In [44]:
# Dependencies
import numpy as np
import pandas as pd
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [18]:
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [19]:
# Create our session (link) from Python to the DB
session = Session(engine)

## D1: Determine the Summary Statistics for June

In [85]:
inspector = inspect(engine)
column_name = []
columns = inspector.get_columns('measurement')
for c in columns:
    column_name.append(c['name'])
    print(c['name'], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [86]:
# Import the sqlalchemy extract function.
from sqlalchemy import extract

# 1. Write a query that filters the Measurement table to retrieve the temperatures for the month of June. 
jun_data = session.query(Measurement.tobs).filter(extract("month",Measurement.date) == "06").all()

In [87]:
# 2. Convert the June temperatures to a list.
jun_data_list = list(np.ravel(jun_data))

In [88]:
# 3. Create a DataFrame from the list of temperatures for the month of June. 
jun_df = pd.DataFrame(jun_data_list,columns=["temperature"])

In [89]:
# 4. Calculate and print out the summary statistics for the June temperature DataFrame.
jun_df.describe()

,temperature
count,1700.000000
mean,74.944118
std,3.257417
min,64.000000
25%,73.000000
50%,75.000000
75%,77.000000
max,85.000000


In [83]:
# 5. Return to your challenge instructions to compare your summary statistics


## D2: Determine the Summary Statistics for December

In [90]:
# 6. Write a query that filters the Measurement table to retrieve the temperatures for the month of December.
dec_data = session.query(Measurement.tobs).filter(func.strftime("%m", Measurement.date) == "12").all()

In [91]:
# 7. Convert the December temperatures to a list.
dec_data_list = list(np.ravel(dec_data))

In [92]:
# 8. Create a DataFrame from the list of temperatures for the month of December. 
dec_df = pd.DataFrame(dec_data_list,columns=["temperature"])

In [93]:
# 9. Calculate and print out the summary statistics for the Decemeber temperature DataFrame.
dec_df.describe()

,temperature
count,1517.000000
mean,71.041529
std,3.745920
min,56.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,83.000000


In [94]:
# 10. Return to your challenge instructions to compare your summary statistics


# # D3: Gather more weather data for June and December

In [154]:
# 11. Diff_Min, Diff_Max and Diff_Mean of temperature between June and December
diff = pd.DataFrame(columns = ['Diff_Min', 'Diff_Max', 'Diff_Mean'])
diff['Diff_Min'] = jun_df.min() - dec_df.min()
diff['Diff_Max'] = jun_df.max() - dec_df.max()
diff['Diff_Mean'] = jun_df.mean() - dec_df.mean()
diff

,Diff_Min,Diff_Max,Diff_Mean
temperature,8.0,2.0,3.902588


In [162]:
# 12. Find data count above the average temperature in June
jun_avg = jun_df.mean()

count_above_avg_jun = session.query(Measurement.date,Measurement.tobs).\
            filter(extract("month",Measurement.date) == "06").\
            filter(Measurement.tobs >= jun_avg).count()
total_count = dec_df['temperature'].count()
print(f'Data Count above average temperature in June: {count_above_avg_jun} out of {total_count}')

Data Count above average temperature in June: 966 out of 1517


In [163]:
# 13. Find the total precitipation in December
dec_prcp = session.query(Measurement.date,Measurement.prcp).\
            filter(func.strftime("%m", Measurement.date) == "12").all()

dec_prcp_df = pd.DataFrame(dec_prcp, columns = ['date', 'precipitation'])

total_prcp_in_dec = dec_prcp_df['precipitation'].sum()

print(f'Total precipitation in December: {total_prcp_in_dec}')

Total precipitation in December: 304.63
